In [89]:
import re, unicodedata, itertools, json
from pathlib import Path
from typing import Dict

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances, silhouette_score
from sklearn.cluster import AgglomerativeClustering

# Input is a folder filled with csv files of tenancy schedules

CFG = {
    "src_dir": Path(r"C:\Users\tim\Documents\Thesis\Programming\Data\SCHEDULES"),    
    "out_dir": Path(r"C:\Users\tim\Documents\Thesis\Programming\Data\processed"),   
}
CFG["out_dir"].mkdir(parents=True, exist_ok=True)

## Utlity helpers
We have 2 helper functions here:

clean: performs ASCII-based normalisation on column names for easier matching.

embed_column: converts a single dataframe column into a vector embedding as described in ALITE using SBERT. (They used a research model here called TURL)  

These embeddings are the foundation for clustering semantically similar columns across different tables.


In [90]:
def clean(text: str) -> str:
    return re.sub(r"[^\w\s]", " ",unicodedata.normalize("NFKC", str(text).lower())).strip()

embedder = SentenceTransformer("all-mpnet-base-v2")

def embed_column(col: pd.Series, col_name: str, sample: int = 50) -> np.ndarray:
    non_null = col.dropna().astype(str)
    if non_null.empty:
        return np.zeros(embedder.get_sentence_embedding_dimension())

    samples = non_null.sample(min(sample, len(non_null)), random_state=0).tolist()
    inputs = [f"{col_name}: {val}" for val in samples]
    vecs = embedder.encode(inputs, show_progress_bar=False)
    return np.mean(vecs, axis=0)



## Schema alignment through clustering columns

This function assigns integration IDS to each column across all input tenancy schedule tables.

STEPS:
- Embed all columns using SBERT with the column name and value as context.
- Compute cosine distance matrix.
- Try to force that no columns from the same table are clustered by setting distances within table to 2.0
- Hierarchical clustering is applied to group similar columns.
- The number of clusters k is chosen using the silhouette score to maximise cluster quality/accuracy. The score indicates how well it merges and can be compared to see how well a column fit into other clusters.
- Columns assigned to the same cluster are given the common integration ID (in final csv they are: iid_000 ,iid_001 etc).

So in this part we try to group semantically similar columns without combining columns from the same schema.


In [91]:
def assign_iids(tables: Dict[str, pd.DataFrame]) -> Dict[str, Dict[str, str]]:
    cols, vecs = [], []
    for tname, df in tables.items():
        for cname in df.columns:
            v = embed_column(df[cname], cname)
            if np.linalg.norm(v) > 1e-6:
                cols.append((tname, cname))
                vecs.append(v)

    X = normalize(np.vstack(vecs))
    D = pairwise_distances(X, metric="cosine")

    tbl = np.array([t for t, _ in cols])
    D[tbl[:, None] == tbl[None, :]] = 2.0    

    min_k = max(len(df.columns) for df in tables.values())
    max_k = len(X) - 1
    
    if min_k == 1:
        best_k, best_s = 1, -1
    else:
        best_k, best_s = min_k, -1
        for k in range(min_k, max_k + 1):
            labels = AgglomerativeClustering(n_clusters=k, metric="precomputed", linkage="average").fit_predict(D)
            s = silhouette_score(X, labels, metric="cosine")
            if s > best_s:
                best_k, best_s = k, s

    labels = AgglomerativeClustering(n_clusters=best_k, metric="precomputed", linkage="average").fit_predict(D)
    lab2iid = {lab: f"iid_{i:03d}" for i, lab in enumerate(np.unique(labels))}
    mapping = {}
    
    for (t, c), lab in zip(cols, labels):
        mapping.setdefault(t, {})[c] = lab2iid[lab]
        
    return mapping


## Outer Union with Null Masking

We align all tenancy schedule tables using the assigned integration IDs and compute the outer union.

- Only the columns with assigned IIDs are preserved.
- If a table lacks a column present in the union, it is padded with a NaN. In the paper they call this a produced Null and it is a important part of why ALITE can cause a more bloated schema and is less business applicable according to this research. 



In [92]:
def outer_union(tables: Dict[str, pd.DataFrame], mapping):
    all_iids = sorted({iid for mp in mapping.values() for iid in mp.values()})
    frames, masks = [], []

    for fname, df in tables.items():
        df = df[[c for c in df.columns if c in mapping[fname]]].copy()
        df.columns = [mapping[fname][c] for c in df.columns]

        df = (df.T.groupby(level=0, sort=False).agg(lambda col: col.bfill().infer_objects(copy=False).iloc[0]).T)
        
        orig_na = df.isna()        
        for iid in all_iids:
            if iid not in df.columns:
                df[iid] = np.nan

        true_na = pd.DataFrame(False, index=df.index, columns=all_iids)
        true_na.loc[:, orig_na.columns] = orig_na
        
        frames.append(df[all_iids])
        masks.append(true_na[all_iids])

    return (pd.concat(frames, ignore_index=True),
            all_iids,
            pd.concat(masks, ignore_index=True))


## Full Disjunction FD

This is the core of the ALITE framework, where we merge overlapping records and remove redundancy.

Null labelling is done by labelling true missing values with placeholders.
Produced nulls are treated as wildcards and empty strings during matching.

During the iterative complementation steps tuples are merged if the overlap atleast >= MIN_OVERLAP non-null values. This helps unify partial records of the same entity while avoiding being over aggressive in mering based on more weak or coincidental matches.

After merging, redundant tuples are dropped if they are completely dominated by more complete ones.

Finally, placeholder labels and wildcards/empty strings are restored to NaN. This tries to yield a more compact and complete set of aligned tuples.


In [93]:
def complement_union(df: pd.DataFrame, iids: list[str], mask: pd.DataFrame):
    MIN_OVERLAP = 3
    uid, df2 = 0, df.copy()
    for iid in iids:
        m = mask[iid]
        df2.loc[m, iid] = [f"NULL{uid+i}" for i in range(int(m.sum()))]
        uid += int(m.sum())

    df2.replace({np.nan: ""}, inplace=True) 
    changed, rows = True, df2[iids].to_numpy().tolist()
    while changed:
        changed, out = False, []
        for t in rows:
            merged = False
            for j, u in enumerate(out):
                common = [k for k in range(len(iids)) if t[k] == u[k] and t[k] != ""]
                if len(common) >= MIN_OVERLAP and all(t[k] == u[k] or t[k] == "" or u[k] == "" for k in range(len(iids))): 
                    out[j] = [t[k] if u[k]=="" else u[k] for k in range(len(iids))]
                    changed = merged = True
                    break
            if not merged:
                out.append(t)
        rows = out

    fd = pd.DataFrame(rows, columns=iids).replace({r"^NULL\d+$": np.nan}, regex=True)
    keep = []
    
    for i, r in fd.iterrows():
        keep.append(not any(
            i!=j and fd.loc[j].isna().sum() < r.isna().sum() and
            all(pd.isna(r[c]) or fd.loc[j,c]==r[c] for c in iids)
            for j in range(len(fd))))
        
    fd.replace("", np.nan, inplace=True)
    return fd[keep]


## Execution
We now run the full pipeline:

Load raw CSVs from disk (tenancy schedules in different formats). Assign IIDs via clustering then Outer-union tables. Run FD to complement and compact tuples. Finally export the standardised table to CSV




In [95]:
tables = {p.name: pd.read_csv(p, dtype=str) for p in CFG["src_dir"].glob("*.csv")}

iid_map = assign_iids(tables)
ou, iids, m_mask = outer_union(tables, iid_map)
fd = complement_union(ou, iids, m_mask)

nulls = fd.isna().sum().sum()
total = fd.size
null_pct = nulls / total

fd.to_csv(CFG["out_dir"] / "fd_result.csv", index=False)

print("Schema width or no of IIDs:  ", len(iids))
print("Final null % overall:   ", f"{null_pct:.1%}")


C:\Users\tim\AppData\Local\Temp\ipykernel_9092\3212223805.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = (df.T.groupby(level=0, sort=False).agg(lambda col: col.bfill().infer_objects(copy=False).iloc[0]).T)
C:\Users\tim\AppData\Local\Temp\ipykernel_9092\3212223805.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = (df.T.groupby(level=0, sort=False).agg(lambda col: col.bfill().infer_objects(copy=False).iloc[0]).T)
C:\Users\tim\AppData\Local\Temp\ipykernel_9092\3212223805.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffil

Schema width or no of IIDs:   89
Final null % overall:    75.9%


'C:\\Users\\tim'